# Εργαστήριο 7
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Επίτρεψη εγγραφής (enable) σε καταχωρητές
Στα παραδείγματα με καταχωρητές που είδαμε ως τώρα, σε κάθε νέο βήμα (step) του ρολογιού (clock), ο καταχωρητής αποθηκεύει ό,τι βρίσκεται στην είσοδό του.

Σε έναν επεξεργαστή συχνά η εγγραφή σε καταχωρητές δεν θέλουμε να γίνεται σε κάθε βήμα του ρολογιού, παρά μόνο αν επιτρέπεται από κάποια συνθήκη. Αυτό ρυθμίζεται από ένα **σήμα επίτρεψης εγγραφής** (`write enable` ή απλά `enable`).

Στην Chisel μπορούμε πολύ εύκολα να επιλέξουμε πότε θα γίνει εγγραφή δεδομένων σε έναν καταχωρητή χρησιμοποιώντας τη δομή `when`. Στο παράδειγμα που ακολουθεί, τα δεδομένα εισόδου εγγράφονται στον καταχωρητή μόνο αν το επιτρέπει το εξωτερικό σήμα `enable`:

In [ ]:
class EnableExample(n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val enable = Input(Bool())
    val out = Output(UInt(n.W))
  })
    
  val testReg = Reg(UInt(n.W))
  
  when(io.enable===true.B) {
    testReg := io.in    
  }
    
  io.out := testReg
}

### Όταν δεν προσδιορίζεται η είσοδος σε καταχωρητή
Στα κυκλώματα χωρίς καταχωρητές η Chisel απαιτεί να προσδιορίζεται πάντα η τιμή ενός σήματος,
αλλιώς παράγεται σφάλμα.

Στο προηγούμενο παράδειγμα με τον καταχωρητή όμως, παρατηρήστε ότι **δεν προσδιορίζεται** η είσοδος στον `testReg`
όταν το `io.enable` δεν είναι αληθές! Γιατί εδώ δεν παράγεται σφάλμα;
Η Chisel θεωρεί ότι στην περίπτωση αυτή απλά θέλετε να διατηρηθεί η τρέχουσα τιμή του καταχωρητή
(όποια κι αν είναι αυτή).

Συνεπώς, **όταν πρόκειται για εισόδους καταχωρητών, μπορείτε να μην προσδιορίσετε πλήρως την τιμή τους**,
χωρίς να παραχθεί σφάλμα από την Chisel.

Δοκιμάστε το προηγούμενο κύκλωμα:

In [ ]:
test(new EnableExample(8)) { c =>
  c.io.enable.poke(false.B)   // δεν επιτρέπεται η εγγραφή στον καταχωρητή
  c.io.in.poke(33.U)
  c.clock.step()
  println(c.io.out.peek())    // το 33 δεν θα εμφανιστεί στην έξοδο του καταχωρητή
    
  c.io.enable.poke(true.B)    // επιτρέπεται η εγγραφή στο επόμενο clock
  c.clock.step()
  println(c.io.out.peek())    // το 33 θα εμφανιστεί στην έξοδο
}

## Άσκηση 1
Κατασκευάστε κύκλωμα με έναν καταχωρητή εύρους `n` bits (παραμετρική υλοποίηση) και τις αντίστοιχες εισόδους-εξόδους:
* `in` (εύρος n bits): είσοδος δεδομένων
* `out` (εύρος n bits): έξοδος δεδομένων

Σε κάθε βήμα (κύκλο) ρολογιού εισάγεται στο κύκλωμα μια νέα τιμή στην είσοδο αλλά **ο καταχωρητής συγκρατεί μόνο τιμές μεγαλύτερες από αυτήν που έχει ως αποθηκευμένη ως τώρα**. Συμπληρώστε το επόμενο κελί.

Συνεπώς ο καταχωρητής θα εμφανίζει κάθε στιγμή στην έξοδο **τη μέγιστη** από τις τιμές που έχουν εμφανιστεί ως τώρα.

Ο καταχωρητής θα πρέπει να είναι αρχικοποιημένος με την τιμή `0.U`.

Θυμηθείτε ότι η Chisel διαθέτει τους γνωστούς τελεστές σύγκρισης:

| τελεστές | έλεγχος |
| --- | --- |
| === | ισότητα |
| =/= | ανισότητα |
| >, >=, <, <= | σύγκριση |

In [ ]:
class MaxRegister(n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val out = Output(UInt(n.W))
  })
    
  // συμπληρώστε...
    
}

Δοκιμάστε τη λειτουργία του κυκλώματός σας με τον παρακάτω κώδικα ελέγχου: 

In [ ]:
test(new MaxRegister(8)) { c =>
  val numbers = List(10,20,5,7,22,9,44,2,43)
  for (number <- numbers) {
    c.io.in.poke(number.U)
    c.clock.step()
  }
  //println(c.io.out.peek())
  c.io.out.expect(numbers.max.U)

}
println("SUCCESS!!")

## Συστοιχία καταχωρητών (register file)
Σε έναν επεξεργαστή οι εντολές μηχανής χρησιμοποιούν έναν αριθμό καταχωρητών για την αποθήκευση προσωρινών αποτελεσμάτων. Η **συστοιχία καταχωρητών** (register file) είναι ένα module που περιέχει τους καταχωρητές αυτούς.

Για να δημιουργήσουμε στην Chisel μια συστοιχία καταχωρητών χρησιμοποιούμε τη σύνταξη:
~~~scala
val regFile = Reg(Vec(m,UInt(n.W)))
~~~
Η πιο πάνω εντολή θα δημιουργήσει `m` καταχωρητές, με εύρος λέξης `n` ο καθένας, π.χ. για να δημιουργήσετε 8 καταχωρητές με εύρος λέξης 32 bits θα γράψετε:
~~~scala
val regFile = Reg(Vec(8,UInt(32.W)))
~~~

Αν θέλετε οι καταχωρητές να έχουν συγκεκριμένη **αρχική τιμή**, η σύνταξη είναι λίγο πιο σύνθετη. Για παράδειγμα, αν θέλετε 8 καταχωρητές με εύρος λέξης 32 bits αρχικοποιημένους στο 0, θα γράψετε:
~~~scala
val regFile = RegInit(VecInit(Seq.fill(8)(0.U(32.W))))
~~~
*Σημ.: η έκφραση `Seq.fill(8)(0.U(n.W))` είναι της υποκείμενης γλώσσας (Scala) και δημιουργεί μια λίστα 8 στοιχείων με τιμή `0.U(32.W)` το καθένα.*

Όταν το `regFile` οριστεί όπως στα δύο προηγούμενα, τότε μπορείτε να διαβάσετε ή να γράψετε τον i-οστό καταχωρητή
της συστοιχίας με τη σύνταξη `regFile(i)`.

### Παράδειγμα register file
Δείτε το επόμενο παράδειγμα υλοποίησης ενός module με register file. Υπάρχουν 8 καταχωρητές, ενώ το εύρος είναι παραμετρικό (`n`).

Περιγραφή εισόδων-εξόδων:
* in (n bits): είσοδος δεδομένων
* out (n bits): έξοδος δεδομένων
* read_select (3 bits): επιλογή καταχωρητή για ανάγνωση, τα δεδομένα του εμφανίζονται στην έξοδο
* write_select (3 bits): επιλογή καταχωρητή για εγγραφή, τα δεδομένα εισόδου θα εγγραφούν σε αυτόν στο επόμενο βήμα ρολογιού.

*Σημ.: τα σήματα select έχουν εύρος 3 bits γιατί οι καταχωρητές είναι 8 (2^3).*

In [ ]:
class RegisterFilePartial(n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val read_select = Input(UInt(3.W))
    val write_select = Input(UInt(3.W))      
    val out = Output(UInt(n.W))
  })
    
  val regFile = Reg(Vec(8,UInt(n.W)))
    
  regFile(io.write_select) := io.in    // εγγραφή δεδομένων
    
  io.out := regFile(io.read_select)    // ανάγνωση δεδομένων
  
}

In [ ]:
test(new RegisterFilePartial(8)) { c =>
  // εγγραφή δεδομένων, r0 = 33, r1 = 34, r2 = 35 κλπ
  for (i <- 0 to 7) {
    c.io.write_select.poke(i.U)
    c.io.in.poke((33+i).U)
    c.clock.step()
  }
  // ανάγνωση δεδομένων από όλους τους καταχωρητές    
  for (i <- 0 to 7) {
    c.io.read_select.poke(i.U)
    println(i,c.io.out.peek())
  }

}

## Άσκηση 2
Τροποποιήστε το προηγούμενο παράδειγμα, έτσι ώστε:

1. Οι καταχωρητές να αρχικοποιούνται στην τιμή 0.
2. Η εγγραφή να πραγματοποιείται μόνο όταν το `write_select` είναι διάφορο του 0. Ο καταχωρητής 0 δηλαδή, **δεν εγγράφεται αλλά έχει πάντα την τιμή 0**.

Συμπληρώστε στο επόμενο κελί:

In [ ]:
class RegisterFile(n: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(n.W))
    val read_select = Input(UInt(3.W))
    val write_select = Input(UInt(3.W))      
    val out = Output(UInt(n.W))
  })
    
  // συμπληρώστε...
    
}

Δοκιμάστε την ορθότητα της υλοποίησής σας με τον επόμενο κώδικα ελέγχου:

In [ ]:
test(new RegisterFile(8)) { c =>

  for (i <- 0 to 7) {
    c.io.write_select.poke(i.U)
    c.io.in.poke((33+i).U)
    c.clock.step()
  }
    
  for (i <- 0 to 7) {
    c.io.read_select.poke(i.U)
    c.io.out.expect( if (i>0) (33+i).U else 0.U)   // εδώ ο r0 θα πρέπει να είναι 0 κι όχι 33
  }

}
println("SUCCESS!!")